# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import math
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from tqdm import tqdm

1. Создать Dataset для загрузки данных (используем только числовые данные)

In [3]:
df_avito = pd.read_csv('/content/drive/MyDrive/train.csv')
df_avito.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [4]:
df_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   item_id               1503424 non-null  object 
 1   user_id               1503424 non-null  object 
 2   region                1503424 non-null  object 
 3   city                  1503424 non-null  object 
 4   parent_category_name  1503424 non-null  object 
 5   category_name         1503424 non-null  object 
 6   param_1               1441848 non-null  object 
 7   param_2               848882 non-null   object 
 8   param_3               640859 non-null   object 
 9   title                 1503424 non-null  object 
 10  description           1387148 non-null  object 
 11  price                 1418062 non-null  float64
 12  item_seq_number       1503424 non-null  int64  
 13  activation_date       1503424 non-null  object 
 14  user_type             1503424 non-

In [5]:
df_avito.isnull().values.any()

True

In [6]:
df_avito = df_avito.fillna(0)

In [7]:
df_avito.isnull().values.any()

False

In [8]:
feature_name = ['price','item_seq_number','image_top_1']
target_name = ['deal_probability']

In [9]:
class MyAvito(torch.utils.data.Dataset):
    def __init__(self, df, feature_name, target_name):

        self.features = df[feature_name]
        self.targets = df[target_name]
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return torch.tensor(self.features.iloc[index]).float(), torch.tensor(self.targets.iloc[index]).float()

In [10]:
ds_avito = MyAvito(df_avito, feature_name, target_name)
ds_avito[0]

(tensor([ 400.,    2., 1008.]), tensor([0.1279]))

In [11]:
ds_train, ds_test = train_test_split(ds_avito, test_size=0.25, random_state=12)
print(len(ds_train), len(ds_test))

1127568 375856


2. Обернуть его в Dataloader

In [17]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=512)

3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

In [18]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x

4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать

In [19]:
def log_RMSE(p, t):
    return torch.sqrt(torch.mean((torch.log(p + 1) - torch.log(t + 1)) ** 2))



5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

In [20]:
net = FeedForward(3, 200)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [21]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(dl_train , 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = log_RMSE(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0
print()
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.001
[1,   301] loss: 0.220
[1,   601] loss: 0.191
[1,   901] loss: 0.191
[1,  1201] loss: 0.190
[1,  1501] loss: 0.191
[1,  1801] loss: 0.191
[1,  2101] loss: 0.191


 10%|█         | 1/10 [00:07<01:04,  7.14s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.190
[2,   601] loss: 0.190
[2,   901] loss: 0.191
[2,  1201] loss: 0.190
[2,  1501] loss: 0.191
[2,  1801] loss: 0.191
[2,  2101] loss: 0.190


 20%|██        | 2/10 [00:13<00:55,  6.94s/it]

...

[9,     1] loss: 0.001
[9,   301] loss: 0.189
[9,   601] loss: 0.189
[9,   901] loss: 0.190
[9,  1201] loss: 0.189
[9,  1501] loss: 0.190
[9,  1801] loss: 0.190
[9,  2101] loss: 0.190


 90%|█████████ | 9/10 [01:03<00:07,  7.25s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.189
[10,   601] loss: 0.189
[10,   901] loss: 0.190
[10,  1201] loss: 0.189
[10,  1501] loss: 0.190
[10,  1801] loss: 0.190
[10,  2101] loss: 0.189


100%|██████████| 10/10 [01:10<00:00,  7.10s/it]


Training is finished!


In [22]:
net = FeedForward(3, 200)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [23]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(dl_train , 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = log_RMSE(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0
print()
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.001
[1,   301] loss: 0.191
[1,   601] loss: 0.188
[1,   901] loss: 0.188
[1,  1201] loss: 0.188
[1,  1501] loss: 0.188
[1,  1801] loss: 0.188
[1,  2101] loss: 0.188


 10%|█         | 1/10 [00:08<01:12,  8.06s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.187
[2,   601] loss: 0.187
[2,   901] loss: 0.188
[2,  1201] loss: 0.188
[2,  1501] loss: 0.188
[2,  1801] loss: 0.188
[2,  2101] loss: 0.187


 20%|██        | 2/10 [00:15<01:02,  7.83s/it]

...

[9,     1] loss: 0.001
[9,   301] loss: 0.187
[9,   601] loss: 0.187
[9,   901] loss: 0.187
[9,  1201] loss: 0.187
[9,  1501] loss: 0.187
[9,  1801] loss: 0.187
[9,  2101] loss: 0.187


 90%|█████████ | 9/10 [01:10<00:07,  7.92s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.187
[10,   601] loss: 0.187
[10,   901] loss: 0.187
[10,  1201] loss: 0.187
[10,  1501] loss: 0.187
[10,  1801] loss: 0.187
[10,  2101] loss: 0.187


100%|██████████| 10/10 [01:18<00:00,  7.87s/it]


Training is finished!


In [24]:
net = FeedForward(3, 200)

optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)

In [25]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(dl_train , 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = log_RMSE(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0
print()
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.001
[1,   301] loss: 0.189
[1,   601] loss: 0.188
[1,   901] loss: 0.189
[1,  1201] loss: 0.188
[1,  1501] loss: 0.188
[1,  1801] loss: 0.188
[1,  2101] loss: 0.188


 10%|█         | 1/10 [00:07<01:07,  7.44s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.188
[2,   601] loss: 0.187
[2,   901] loss: 0.188
[2,  1201] loss: 0.188
[2,  1501] loss: 0.188
[2,  1801] loss: 0.188
[2,  2101] loss: 0.188


 20%|██        | 2/10 [00:14<00:59,  7.44s/it]

...

[9,     1] loss: 0.001
[9,   301] loss: 0.187
[9,   601] loss: 0.187
[9,   901] loss: 0.188
[9,  1201] loss: 0.187
[9,  1501] loss: 0.187
[9,  1801] loss: 0.188
[9,  2101] loss: 0.188


 90%|█████████ | 9/10 [01:07<00:07,  7.47s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.187
[10,   601] loss: 0.187
[10,   901] loss: 0.188
[10,  1201] loss: 0.187
[10,  1501] loss: 0.187
[10,  1801] loss: 0.188
[10,  2101] loss: 0.187


100%|██████████| 10/10 [01:14<00:00,  7.46s/it]


Training is finished!


Наиболее эффективно показал себя оптимизатор Адам. Он быстрее всех сошёлся и потери постояно составляли 0.187